In [1]:
import numpy as np
from qore import Mine, ASP, QAOA, VQE
from qore.utils import measure_operator, get_bitstring_probabilities
from qore.algorithms.asp import construct_default_H_B
from qiskit.aqua import aqua_globals, QuantumInstance
from qiskit import Aer

In [2]:
penalty = 5.0
qmine = Mine(np.array([[-2.0, 3.0, -1.0, -2.0, -1.0], [float('inf'), 1.0, -5.0, 10.0, float('inf')], [float('inf'), float('inf'), 4.0, float('inf'), float('inf')]]))
#qmine = Mine('mine_config.txt')
 
qmine.plot_mine()

+---+--------+-------+--------+--------+--------+
|   |   0    |   1   |   2    |   3    |   4    |
+---+--------+-------+--------+--------+--------+
| 0 | -2.000 | 3.000 | -1.000 | -2.000 | -1.000 |
| 1 |  inf   | 1.000 | -5.000 | 10.000 |  inf   |
| 2 |  inf   |  inf  | 4.000  |  inf   |  inf   |
+---+--------+-------+--------+--------+--------+


In [3]:
H = qmine.gen_Hamiltonian(penalty=penalty)

In [4]:
aqua_globals.random_seed = 1953
backend = Aer.get_backend('statevector_simulator')
instance = QuantumInstance(backend=backend)

In [6]:
def analysis(circuit):
    x = get_bitstring_probabilities(circuit, instance, shots=1024)
    # x_sorted = sorted(x.items(), key=lambda item: item[1])
    # for i in range(5):
    #     print("The most probable configurations and the corresponding probabilities:")
    #     print(f"{x_sorted[i][0]}, {x_sorted[i][1]}")
    bitstr, prob = max(x.items(), key=lambda item: item[1])
    print(f"The most probable configuration and the corresponding probability: {bitstr, prob}")
    print("------------------------------------------------------------")


In [7]:
evol_time = 20
nsteps = 100

circuit = ASP(H, 
    evol_time=evol_time, 
    nsteps=nsteps, 
    callback=analysis,
    callback_freq=40,
    quantum_instance=instance).construct_circuit()
    

#analysis(circuit)

The most probable configuration and the corresponding probability: ('101010100', 0.0068359375)
------------------------------------------------------------
The most probable configuration and the corresponding probability: ('010011110', 0.220703125)
------------------------------------------------------------
The most probable configuration and the corresponding probability: ('010011110', 0.94140625)
------------------------------------------------------------


In [8]:
s = '010011110'
qmine.plot_mine_state(s)

+---+---+---+---+---+---+
|   | 0 | 1 | 2 | 3 | 4 |
+---+---+---+---+---+---+
| 0 | 0 | 1 | 0 | 0 | 1 |
| 1 | x | 1 | 1 | 1 | x |
| 2 | x | x | 0 | x | x |
+---+---+---+---+---+---+


In [9]:
fs = ''.join(list(s)[::-1])
qmine.plot_mine_state(fs)

+---+---+---+---+---+---+
|   | 0 | 1 | 2 | 3 | 4 |
+---+---+---+---+---+---+
| 0 | 0 | 1 | 1 | 1 | 1 |
| 1 | x | 0 | 0 | 1 | x |
| 2 | x | x | 0 | x | x |
+---+---+---+---+---+---+
